# 第五章 例5.6.1  用直接交叉熵平衡SAM表

In [297]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Domain,Sense
from gamspy.math import log
import pandas as pd
pd.options.display.float_format = '{:.1f}'.format

### Container

In [298]:
m = Container()

### Sets

In [299]:
i = Set(container=m, name="i", description="units", records=["sec1","sec2","lab","hh"])
j = Alias(m, name="j", alias_with=i)

### Data

In [300]:
data = pd.read_excel("ch5-5-1.xlsx", index_col=0)
data = data.iloc[:-2,:-2]
data = data.stack().reset_index()
data


,level_0,level_1,0
0,sec1,sec1,52.0
1,sec1,sec2,45.0
2,sec1,hh,150.0
3,sec2,sec1,95.0
4,sec2,sec2,48.0
5,sec2,hh,90.0
6,lab,sec1,120.0
7,lab,sec2,89.0
8,hh,lab,192.0


### Parameters

In [301]:
Q0 = Parameter(
    container=m,
    name="Q0",
    domain=[i,j],
    description="intial value",
    records=data
)
Q0.records


,level_0,level_1,value
0,sec1,sec1,52.0
1,sec1,sec2,45.0
2,sec1,hh,150.0
3,sec2,sec1,95.0
4,sec2,sec2,48.0
5,sec2,hh,90.0
6,lab,sec1,120.0
7,lab,sec2,89.0
8,hh,lab,192.0


In [302]:
H0 = Parameter(
    container=m,
    name="H0",
    description="intial sum value",
    records=data.iloc[:,2].sum()
)
H0.records

,value
0,881.0


### Variables

In [303]:
Q = Variable(container=m, name="Q", domain=[i,j],type="Positive", description="adjusted value")
H = Variable(container=m, name="H", description="sum of adjusted value")
Hratio = Variable(container=m, name="Hratio", description="ratio of sum of adjusted value")

### Equations

In [304]:
totalsum = Equation(container=m, name="totalsum",description="total sum equation" )
totalsum[...] = H == Sum(Domain(i,j).where[Q0[i,j]>0], Q[i,j])

Hratiodef = Equation(container=m, name="Hratiodef",description="H ratio equation" )
Hratiodef[...] = Hratio == H/H0

balance = Equation(container=m, name="balance", domain=i,description="balance equation" )
balance[i] = Sum(j.where[Q0[i,j]>0], Q[i,j])  == Sum(j.where[Q0[j,i]>0], Q[j,i])

### Objective

In [305]:
obj = Sum(Domain(i, j).where[Q0[i,j]>0],(1/H)*Q[i,j]*log(Q[i,j]/Q0[i,j])-log(Hratio) )

### Model

In [306]:
sambal = Model(m, name="sambal", equations=[totalsum,Hratiodef,balance],problem="NLP", sense=Sense.MIN, objective=obj)
Q.l[i,j] = Q0[i,j]
H.l[...] = H0
Hratio.lo[...] = 0.5
Hratio.up[...] = 2

In [307]:
sambal.solve()
Q.records

,i,j,level,marginal,lower,upper,scale
0,sec1,sec1,104.7,0.0,0.0,inf,1.0
1,sec1,sec2,106.8,0.0,0.0,inf,1.0
2,sec1,hh,285.3,0.0,0.0,inf,1.0
3,sec2,sec1,162.3,0.0,0.0,inf,1.0
4,sec2,sec2,96.7,0.0,0.0,inf,1.0
5,sec2,hh,145.2,0.0,0.0,inf,1.0
6,lab,sec1,229.7,0.0,0.0,inf,1.0
7,lab,sec2,200.8,0.0,0.0,inf,1.0
8,hh,lab,430.5,0.0,0.0,inf,1.0


In [308]:
H.records

,level,marginal,lower,upper,scale
0,1762.0,-0.0,-inf,inf,1.0


In [309]:
Hratio.records

,level,marginal,lower,upper,scale
0,2.0,-4.0,0.5,2.0,1.0


In [310]:
sambal.objective_value

-5.538425454783181

In [311]:
results = Q.records.loc[:,["i","j","level"]].pivot(index="i",columns="j",values="level")
results

j,sec1,sec2,lab,hh
i,,,,
sec1,104.7,106.8,NaN,285.3
sec2,162.3,96.7,NaN,145.2
lab,229.7,200.8,NaN,NaN
hh,NaN,NaN,430.5,NaN


In [312]:
results.sum()

j
sec1   496.8
sec2   404.2
lab    430.5
hh     430.5
dtype: float64

In [313]:
results.sum(1)

i
sec1   496.8
sec2   404.2
lab    430.5
hh     430.5
dtype: float64

## 验证书中的表格

In [314]:
# 原始数据
import numpy as np
data = pd.read_excel("ch5-5-1.xlsx", index_col=0)
data = data.iloc[:-2,:-2].values 
data

array([[ 52.,  45.,  nan, 150.],
       [ 95.,  48.,  nan,  90.],
       [120.,  89.,  nan,  nan],
       [ nan,  nan, 192.,  nan]])

In [315]:
# 书中最优化表格
data1 = pd.read_excel("ch5-6-1.xlsx", index_col=0)
data1 = data1.values
data1

array([[ 52.2,  53.2,   nan, 142.2],
       [ 80.9,  48.2,   nan,  72.4],
       [114.5, 100.1,   nan,   nan],
       [  nan,   nan, 214.6,   nan]])

In [316]:
np.nansum(data1,0)

array([247.6, 201.5, 214.6, 214.6])

In [317]:
np.nansum(data1,1)

array([247.6, 201.5, 214.6, 214.6])

In [318]:
H0 = np.nansum(data)
H = np.nansum(data1)
Hratio = H / H0
H0, H, Hratio

(881.0, 878.3000000000001, 0.9969353007945517)

In [319]:
z = (1/H)*np.nansum(data1*np.log(data1/data))-np.log(H/H0)
z

0.006752007506542378